In [13]:
pwd


'/home/wsuser/work'

In [6]:
cd /home/wsuser/work

/home/wsuser/work


In [14]:
!pip install keras
!pip install tensorflow

In [32]:
pwd

'/home/wsuser/work'

In [28]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten

In [25]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ccAqZx_IM2IAQrD_b0-CuU1QCMZDzMqqYM4N7oIRB-Wu',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'naturaldisasterintensityanalysisa-donotdelete-pr-pkpliyi4wdyll3'
object_key = 'dataset.zip'

streaming_body_10 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [ ]:
from io import BytesIO
import zipfile
unzip= zipfile.ZipFile(BytesIO(streaming_body_10.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [34]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [35]:
train_datagen=ImageDataGenerator(rescale=1./255,zoom_range=0.2,horizontal_flip=True,shear_range=0.2)

In [36]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [1]:
x_train=train_datagen.flow_from_directory(r"/home/wsuser/work/dataset/train_set",target_size=(64,64),
                                                            batch_size=5,color_mode='rgb',class_mode='categorical')

NameError: name 'train_datagen' is not defined

In [2]:
x_test=test_datagen.flow_from_directory(r"/home/wsuser/work/dataset/test_set",target_size=(64,64),
                                                            batch_size=5,color_mode='rgb',class_mode="categorical")

NameError: name 'test_datagen' is not defined

In [ ]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten

In [ ]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation="relu",input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=4,activation='softmax'))
model.compile(loss="categorical_crossentropy",metrics=["accuracy"],optimizer='adam')

In [ ]:
model.summary()

In [ ]:
model.fit_generator(generator=x_train,epochs=20,steps_per_epoch=len(x_train),validation_data=x_test,validation_steps=len(x_test))

In [ ]:
model.save('disaster.h5')
model_json=model.to_json()
with open("model-bw.json","w") as json_file:
  json_file.write(model_json) 

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
model=load_model('disaster.h5')

In [3]:
x_train.class_indices

NameError: name 'x_train' is not defined

In [ ]:
img=image.load_img(r"/content/drive/MyDrive/Disaster/dataset/test_set/Earthquake/1329.jpg",target_size=(64,64))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
index=['Cyclone','Earthquake','Flood','Wildfire']
y=np.argmax(model.predict(x),axis=1)
print(index[int(y)])

In [ ]:
img=image.load_img(r"/content/drive/MyDrive/Disaster/dataset/test_set/Cyclone/900.jpg",target_size=(64,64))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
index=['Cyclone','Earthquake','Flood','Wildfire']
y=np.argmax(model.predict(x),axis=1)
print(index[int(y)])